In [1]:
# Use autoplot 207's code for this app
import sys
import datetime
sys.path.insert(0, "/opt/iem/htdocs/plotting/auto/scripts200/")
from p207 import load_data, compute_grid_bounds, add_zeros, do_analysis, USEME
from matplotlib.patches import Rectangle
from shapely.geometry import Point
from pyiem.plot import MapPlot, nwssnow
from pyiem.nws.vtec import NWS_COLORS
from pyiem.util import get_dbconn, utc, get_sqlalchemy_conn
from pyiem.reference import EPSG
from pyproj import Transformer
import numpy as np
import pandas as pd
import geopandas as gpd

In [16]:
STORM_NUMBER = 19
TITLE = "22-23 February 2023"
SUBTITLE = "8 AM 23 February 2023"
SETPOINT_LOCS = {}
sts = utc(2023, 2, 22, 18)
ets = utc(2023, 2, 24, 20)
# Get available data
ctx = {'coop': 'yes', 't': 'state', 'sz': 30, 'z': 'no', 'f': 'linear', 'v': 'ice', 'wfo': 'DMX'}
df = load_data(ctx, sts, ets)
# figure out our grid bounds
ctx["bnds2163"] = compute_grid_bounds(ctx, 'IA')
# add zeros and QC
df = add_zeros(df, ctx)

In [21]:
df[df["wfo"] == "DMX"]

,state,wfo,val,lon,lat,geo,used_for_analysis,nwsli,plotme,source,xcell,ycell
35,IA,DMX,0.50,-93.74,41.94,POINT (517336.500 -320479.349),True,35,True,LSR,8,7
55,IA,DMX,0.40,-92.35,42.32,POINT (627996.359 -268633.455),True,55,True,LSR,12,9
78,IA,DMX,0.30,-93.63,42.03,POINT (525649.478 -309796.903),False,78,False,LSR,8,7
100,IA,DMX,0.25,-93.69,42.02,POINT (520793.665 -311288.851),False,100,False,LSR,8,7
101,IA,DMX,0.25,-93.68,41.95,POINT (522199.669 -318990.044),False,101,False,LSR,8,7
102,IA,DMX,0.25,-92.31,42.17,POINT (632792.453 -284948.431),True,102,True,LSR,12,8
103,IA,DMX,0.25,-92.27,42.55,POINT (632183.752 -242529.240),True,103,True,LSR,12,10
152,IA,DMX,0.20,-92.12,42.85,POINT (641260.821 -208117.669),True,152,True,LSR,12,11
164,IA,DMX,0.20,-93.63,42.02,POINT (525733.424 -310906.120),False,164,False,LSR,8,7
205,IA,DMX,0.15,-93.65,42.03,POINT (524003.194 -309924.867),False,205,False,LSR,8,7


In [30]:
def overlay_ice(mp):
    """Add plotted ice storm."""
    with get_sqlalchemy_conn("postgis") as conn:
        df = pd.read_sql(
        """
        SELECT st_x(geom) as lon, st_y(geom) as lat, magnitude from lsrs WHERE
        typetext in ('ICE STORM', 'FREEZING RAIN') and magnitude > 0
        and valid > %s and valid < %s and state = 'IA'
        """,
        conn,
        params=(sts - datetime.timedelta(days=1), ets)
        )
    print(df)
    mp.plot_values(df.lon.values, df.lat.values, df.magnitude.values, fmt="%.2f", labelbuffer=1, color='purple')

def workflow(ctx, df, isfinal=False, lower=0, upper=2):
    # do gridding
    df2 = df[df[USEME]]
    lons, lats, vals = do_analysis(df2, ctx)
    mp = MapPlot(
        sector="state",
        state=ctx["csector"],
        axisbg="white",
        title="%s - Freezing Rain / Ice Storm Reports" % (TITLE, ),
        subtitle=("From NWS Local Storm Reports; "
                  f"IEM 2022-2023 Winter Storm #{STORM_NUMBER}"),
        twitter=True,
    )
    #mp.contourf(
    #    lons, lats, vals, np.array([0.01, 0.05, 0.1, 0.2, 0.3, 0.5, 0.75, 1]), cmap=nwssnow(), clip_on=True)
    df_useme_plot = df2[(df2['val']>=lower)&(df2['val'] < upper)]
    print(df2)
    mp.drawcounties()
    # overlay_ice(mp)
    if isfinal:
        mp.drawcities()
    else:
        mp.plot_values(df['lon'], df['lat'], df['val'].values,
            '%s', # labels=df_useme_plot['nwsli'].values,
            textsize=10, labeltextsize=10, labelbuffer=1, color='b')
    return mp

In [31]:
def add_setpoints(setpoints):
    """Manual things."""
    for sp, val in setpoints:
        df.at[10000 + sp, "geo"] = Point(SETPOINT_LOCS[sp][0], SETPOINT_LOCS[sp][1])
        df.at[10000 + sp, "val"] = val
        df.at[10000 + sp, USEME] = True
        df.at[10000 + sp, "plotme"] = True

def draw_setpoints(mp):
    """Add some points where manual obs could be inserted."""
    xlim = mp.panels[0].ax.get_xlim()
    ylim = mp.panels[0].ax.set_ylim()
    sz = ctx["sz"] * 1000.
    i = 0
    trans = Transformer.from_proj(mp.panels[0].crs, 2163, always_xy=True)
    for y in np.arange(ylim[0] + sz/2, ylim[1], sz):
        for x in np.arange(xlim[0] + sz/2, xlim[1], sz):
            mp.panels[0].ax.text(x, y, f"{i}", ha="center", va="center")
            # Need to store the x, y in 2163, which is what p207 uses :/
            (xx, yy) = trans.transform(x, y)
            SETPOINT_LOCS[i] = [xx, yy]
            i += 1

def plotsqw(mp):
    with get_sqlalchemy_conn("postgis") as conn:
        gdf = gpd.read_postgis(
            f"SELECT geom from sbw_{sts.year} w WHERE w.phenomena = 'SQ' and w.issue > %s and w.issue < %s",
            conn,
            params=(sts, ets)
        )
    gdf.to_crs(mp.panels[0].crs).plot(
        ax=mp.panels[0].ax,
        aspect=None,
        edgecolor=NWS_COLORS["SQ.W"],
        facecolor="None",
        zorder=1000,
        linewidth=2,
    )
    p0 = Rectangle((0, 0), 1, 1, ec=NWS_COLORS["SQ.W"], fc="None")
    mp.panels[0].ax.legend((p0, ), ("Snow Squall Warning", ), loc=1).set_zorder(1000)

In [32]:
def main():
    setpoints = [
    ]
    if setpoints:
        add_setpoints(setpoints)
    cull = [
    ]
    if cull:
        df.loc[df['nwsli'].isin(cull), USEME] = False
    hardcode = [

    ]
    for nwsli, val in hardcode:
        df.loc[df['nwsli'] == nwsli, 'val'] = val

    ctx['csector'] = "IA"
    mp = workflow(ctx, df, isfinal=False, lower=0, upper=12)
    # draw_setpoints(mp)

    # plotsqw(mp)
    res = mp.postprocess(filename='230224.png')
    mp.close()

main()

    state  wfo   val    lon    lat                              geo  \
0      WI  MKX  6.20 -87.82  43.57    POINT (976796.175 -85772.514)   
1      MI  GRR  0.76 -84.60  42.26  POINT (1258431.682 -185775.142)   
2      WI  MKX  0.75 -87.91  42.75   POINT (982898.152 -177295.255)   
3      WI  MKX  0.73 -88.27  42.68   POINT (954976.836 -189292.169)   
4      MI  GRR  0.70 -85.26  42.37  POINT (1203103.514 -183645.864)   
..    ...  ...   ...    ...    ...                              ...   
301    MA  BOX  0.00 -71.17  42.18   POINT (2315215.205 102034.443)   
304    NY  BUF  0.00 -78.28  43.37    POINT (1729941.057 51757.470)   
306    CT  BOX  0.00 -72.18  41.75    POINT (2253806.069 27714.702)   
307    MI  GRR  0.00 -86.23  43.17  POINT (1110256.191 -109441.036)   
308    NY  BUF  0.00 -76.05  43.59   POINT (1894336.638 125569.769)   

     used_for_analysis  nwsli  plotme source  xcell  ycell  
0                 True      0    True    LSR     23     15  
1                 True   

/opt/miniconda3/envs/prod/lib/python3.10/site-packages/shapely/predicates.py:798: RuntimeWarning: invalid value encountered in intersects
  return lib.intersects(a, b, **kwargs)
